In [13]:
# Taken from https://www.kaggle.com/code/suvroo/complete-nlp-pipeline#RoPE-(Robust-Positional-Embeddings)

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix
import re
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


df = pd.read_csv('IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [14]:
df.duplicated().sum()

418

In [15]:
df.drop_duplicates(inplace=True)

In [16]:

def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text
df['review'] = df['review'].apply(remove_tags)

sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

df['review'] = df['review'].apply(lambda x:x.lower())

In [17]:
# df = df.sample(1000)

In [18]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [19]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
X_train.shape

(39665, 1)

In [21]:
# Applying BoW
cv = CountVectorizer()
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

X_train_bow.shape

(39665, 94725)

In [22]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()
# 
# rf.fit(X_train_bow,y_train)
# y_pred = rf.predict(X_test_bow)
# accuracy_score(y_test,y_pred)

In [23]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [24]:
import optuna
import numpy as np
import xgboost as xgb
from sklearn.model_selection import RepeatedKFold

from sklearn.metrics import roc_auc_score

import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000, step=50),
        "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),  # Learning rate
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),  # L2 regularization
        "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),  # L1 regularization
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 5),  # Minimum loss reduction
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "verbosity": 0,
        "tree_method": "hist",  # Enable GPU
        "device": "cuda",  # Specify CUDA device
    }

    # Train-validation split
    X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(
        X_train_bow, y_train, test_size=0.2, random_state=42
    )

    dtrain = xgb.DMatrix(X_train_split, label=y_train_split)
    dvalid = xgb.DMatrix(X_valid_split, label=y_valid_split)

    model = xgb.train(
        params=params,
        dtrain=dtrain,
        evals=[(dvalid, "eval")],
        early_stopping_rounds=50,
        verbose_eval=False,
    )

    # Predict on validation set
    preds = model.predict(dvalid, iteration_range=(0, model.best_iteration))

    # Compute roc_auc score
    return roc_auc_score(y_valid_split, preds)


# Run Optuna optimization
pruner = optuna.pruners.HyperbandPruner()
study = optuna.create_study(direction="maximize",pruner=pruner)
study.optimize(objective,timeout=3600)

# Best hyperparameters found
best_params = study.best_params
print(f"\n🔹 Best Hyperparameters:\n{best_params}")

# Train final model with best hyperparameters on GPU
best_params["tree_method"] = "gpu_hist"
best_params["device"] = "cuda"

dtrain_cv = xgb.DMatrix(X_train_bow, label=y_train)
final_model = xgb.train(
    params=best_params,
    dtrain=dtrain_cv,
)

# Evaluate on Hold-Out Validation Set
dvalid = xgb.DMatrix(X_test_bow, label=y_test)
test_preds = final_model.predict(dvalid)
accuracy_score(y_test,test_preds)

[I 2025-03-17 17:03:19,849] A new study created in memory with name: no-name-0a6c6a46-1061-4a1a-9788-37b947ee2cbe
[W 2025-03-17 17:10:51,457] Trial 0 failed with parameters: {'n_estimators': 350, 'eta': 0.012804579707508321, 'max_depth': 5, 'subsample': 0.5878476694783148, 'colsample_bytree': 0.5586003874922127, 'lambda': 0.0024288684911069275, 'alpha': 0.04773470416098507, 'min_child_weight': 2, 'gamma': 2.968768279522969} because of the following error: XGBoostError('[17:10:51] C:\\buildkite-agent\\builds\\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\\xgboost\\xgboost-ci-windows\\src\\tree\\updater_gpu_hist.cu:781: Exception in gpu_hist: [17:10:51] C:\\buildkite-agent\\builds\\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\\xgboost\\xgboost-ci-windows\\src\\data\\../common/device_helpers.cuh:431: Memory allocation error on worker 0: bad allocation: cudaErrorMemoryAllocation: out of memory\n- Free memory: 6825181184\n- Requested memory: 9104156256\n\n')

XGBoostError: [17:10:51] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\tree\updater_gpu_hist.cu:781: Exception in gpu_hist: [17:10:51] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\data\../common/device_helpers.cuh:431: Memory allocation error on worker 0: bad allocation: cudaErrorMemoryAllocation: out of memory
- Free memory: 6825181184
- Requested memory: 9104156256

